In [58]:
import numpy as np
from astropy.io import fits
from astropy.table import Table, Column, join
from astropy.wcs import WCS

In [59]:
def fitsextract(filename, stride=[1,1,1], keepref=True, keepnan=True):
    hdu   = fits.open(filename)[0]
    data  = hdu.data
    bunit = hdu.header['BUNIT']
    w     = WCS(hdu)
    print('RA ref is',w.wcs.crval[0])
    print('DEC ref is',w.wcs.crval[1])
    ndim  = len(data.shape)
    iscube = (ndim > 2 and data.shape[ndim-3] > 1)
    if iscube:
        print('This is a data cube')
        data = np.squeeze(data)
        if len(data.shape) > 3:
            raise ('Data cannot be squeezed to three dimensions')
        naxis = 3
        ix,iy,iz = np.meshgrid(np.arange(data.shape[2]),np.arange(data.shape[1]),
                             np.arange(data.shape[0]),indexing='ij')
        tab = Table([np.ravel(ix),np.ravel(iy),np.ravel(iz)],
                  names=('ix','iy','iz'),dtype=('i4','i4','i4'))
        # Get the pixel coordinates as tuples
        wcsin = (np.array([tab['ix'],tab['iy'],tab['iz']])).T
    else:
        print('This is an image')
        data = np.squeeze(data)
        naxis = 2
        ix,iy = np.meshgrid(np.arange(data.shape[1]),np.arange(data.shape[0]),
                            indexing='ij')
        tab = Table([np.ravel(ix),np.ravel(iy)],
                    names=('ix','iy'),dtype=('i4','i4'))
        # Get the pixel coordinates as tuples
        wcsin = (np.array([tab['ix'],tab['iy']])).T
    wfix = w.sub(naxis)
    wcsout = wfix.wcs_pix2world(wcsin,0)
    col_ra = Column(wcsout.T[0]-w.wcs.crval[0], name='ra_off',  dtype='f4', unit='deg')
    col_dc = Column(wcsout.T[1]-w.wcs.crval[1], name='dec_off', dtype='f4', unit='deg')
    tab.add_columns([col_ra,col_dc])
    if iscube:
        col_vel = Column(wcsout.T[2]/1000., name='vel', dtype='f4', unit='km/s')
        tab.add_column(col_vel)
    # Use order = 'F' because the velocity axis is first
    col_data = Column(np.ravel(data,order='F'), name='imgdata', dtype='f4', unit=bunit)
    tab.add_column(col_data)
    # Select the desired rows from the full table
    idx = ['ix', 'iy', 'iz']
    rem = [0, 0, 0]
    select = [[],[],[]]
    if keepref:
        for i in range(naxis):
            crpix = wfix.wcs.crpix[i]
            if crpix < 1 or crpix > data.shape[naxis-i-1] or not crpix.is_integer():
                print('Cannot use keepref on axis {}: crpix={}'.format(i+1,crpix))
                continue
            else:
                print('Axis {}: crpix={}'.format(i+1,crpix))
                rem[i] = int(crpix-1) % stride[i]
        print('Remainder: ',rem)
    for i in range(naxis):
        select[i]=np.where(tab[idx[i]] % stride[i] == rem[i])[0]
    xy = np.intersect1d(select[0], select[1])
    if iscube:
        xyz = np.intersect1d(xy, select[2])
        if len(xyz) < len(tab):
            newtab = tab[xyz]
            tab = newtab
    else:
        if len(xy) < len(tab):
            newtab = tab[xy]
            tab = newtab
    # Remove NaN rows if desired
    if not keepnan:
        newtab = tab[~np.isnan(tab['imgdata'])]
        tab = newtab
    return tab

In [61]:
tab0 = fitsextract('fitsdata/NGC4047.co.smo7_dil.mom0.fits.gz', keepnan=True, stride=[3,3,1])
tab0['imgdata'].name = 'mom0'
print(tab[20:50])
etab0 = fitsextract('fitsdata/NGC4047.co.smo7_dil.emom0.fits.gz', keepnan=True, stride=[3,3,1])
etab0['imgdata'].name = 'emom0'
join1 = join(tab0, etab0)
#join1.show_in_notebook()
join1.write('NGC4047.co.smo7_dil.hdf5', path='data', overwrite=True, serialize_meta=True, compression=True)

RA ref is 180.71125
DEC ref is 48.6361944444
This is an image
Axis 1: crpix=81.0
Axis 2: crpix=81.0
Remainder:  [2, 2, 0]
 ix  iy  iz   ra_off     dec_off     vel   imgdata
               deg         deg      km / s    K   
--- --- --- ---------- ------------ ------ -------
  2   5  15 0.03277313 -0.020837985 3280.0     nan
  2   5  18 0.03277313 -0.020837985 3340.0     nan
  2   5  21 0.03277313 -0.020837985 3400.0     nan
  2   5  24 0.03277313 -0.020837985 3460.0     nan
  2   5  27 0.03277313 -0.020837985 3520.0     nan
  2   5  30 0.03277313 -0.020837985 3580.0     nan
  2   5  33 0.03277313 -0.020837985 3640.0     nan
  2   5  36 0.03277313 -0.020837985 3700.0     nan
  2   5  39 0.03277313 -0.020837985 3760.0     nan
  2   5  42 0.03277313 -0.020837985 3820.0     nan
... ... ...        ...          ...    ...     ...
  2   8  30 0.03277367 -0.020004652 3580.0     nan
  2   8  33 0.03277367 -0.020004652 3640.0     nan
  2   8  36 0.03277367 -0.020004652 3700.0     nan
  2   8  39

Exception: h5py is required to read and write HDF5 files

In [62]:
tab = fitsextract('../img_cocube/fitsdata/NGC4047.co.smo7msk.K.fits.gz', keepnan=True, stride=[3,3,3])
print(tab[20:50])
#tab.write('test.hdf5', path='data', overwrite=True, serialize_meta=True, compression=True)

RA ref is 180.71125
DEC ref is 48.6361944444
This is a data cube
Axis 1: crpix=81.0
Axis 2: crpix=81.0
Axis 3: crpix=1.0
Remainder:  [2, 2, 0]
 ix  iy  iz   ra_off     dec_off     vel   imgdata
               deg         deg      km / s    K   
--- --- --- ---------- ------------ ------ -------
  2   5  15 0.03277313 -0.020837985 3280.0     nan
  2   5  18 0.03277313 -0.020837985 3340.0     nan
  2   5  21 0.03277313 -0.020837985 3400.0     nan
  2   5  24 0.03277313 -0.020837985 3460.0     nan
  2   5  27 0.03277313 -0.020837985 3520.0     nan
  2   5  30 0.03277313 -0.020837985 3580.0     nan
  2   5  33 0.03277313 -0.020837985 3640.0     nan
  2   5  36 0.03277313 -0.020837985 3700.0     nan
  2   5  39 0.03277313 -0.020837985 3760.0     nan
  2   5  42 0.03277313 -0.020837985 3820.0     nan
... ... ...        ...          ...    ...     ...
  2   8  30 0.03277367 -0.020004652 3580.0     nan
  2   8  33 0.03277367 -0.020004652 3640.0     nan
  2   8  36 0.03277367 -0.020004652 3700.